In [1]:
import getpass
import sys
import time

import numpy as np
from copy import deepcopy

from utils import calculate_perplexity, get_ptb_dataset, Vocab
from utils import ptb_iterator, sample

import tensorflow as tf
from tensorflow.python.ops.seq2seq import sequence_loss
from model import LanguageModel

In [2]:
class Config(object):
  """Holds model hyperparams and data information.

  The config class is used to store various hyperparameters and dataset
  information parameters. Model objects are passed a Config() object at
  instantiation.
  """
  batch_size = 64
  embed_size = 50
  hidden_size = 100
  num_steps = 10
  max_epochs = 16
  early_stopping = 2
  dropout = 0.9
  lr = 0.001

In [3]:
config = Config()

In [5]:
vocab = Vocab()

In [6]:
vocab.construct(get_ptb_dataset('train'))

929589.0 total words with 10000 uniques


In [10]:
encoded_train = np.array([vocab.encode(word) for word in get_ptb_dataset('train')], dtype=np.int32)
encoded_valid = np.array([vocab.encode(word) for word in get_ptb_dataset('valid')], dtype=np.int32)
encoded_test  = np.array([vocab.encode(word) for word in get_ptb_dataset('test')], dtype=np.int32)

In [14]:
len(encoded_test)

82430

## Placeholders

In [16]:
input_placeholder   = tf.placeholder(tf.int32, [None, config.num_steps], name="input_placeholder")
labels_placeholder  = tf.placeholder(tf.int32, [None, config.num_steps], name="labels_placeholder")
dropout_placeholder = tf.placeholder(tf.float32, [], name="dropout_placeholder")

## Embeddings

In [18]:
L = tf.Variable(tf.random_uniform([len(vocab), config.embed_size], -1, 1), name="L")

In [20]:
print(L)

Tensor("L/read:0", shape=(10000, 50), dtype=float32)


In [22]:
embeds = tf.nn.embedding_lookup(L, input_placeholder)

In [23]:
embeds

<tf.Tensor 'embedding_lookup_1:0' shape=(?, 10, 50) dtype=float32>

In [25]:
tf.split(split_dim=1, num_split=config.num_steps, value=embeds)

[<tf.Tensor 'split:0' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:1' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:2' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:3' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:4' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:5' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:6' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:7' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:8' shape=(?, 1, 50) dtype=float32>,
 <tf.Tensor 'split:9' shape=(?, 1, 50) dtype=float32>]

In [27]:
inputs = tf.unpack(embeds, axis=1)

In [28]:
inputs

[<tf.Tensor 'unpack_1:0' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:1' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:2' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:3' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:4' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:5' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:6' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:7' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:8' shape=(?, 50) dtype=float32>,
 <tf.Tensor 'unpack_1:9' shape=(?, 50) dtype=float32>]

## Model

In [29]:
initial_state = tf.Variable(tf.zeros([config.batch_size, config.hidden_size]), name="initial_state")

In [52]:
with tf.variable_scope("RNNffebraze") as scope:
    H  = tf.get_variable("H", initializer=tf.random_uniform([config.hidden_size, config.hidden_size], -1, 1))
    I  = tf.get_variable("I", initializer=tf.random_uniform([config.embed_size, config.hidden_size], -1, 1))
    b1 = tf.get_variable("b1", initializer=tf.zeros([config.hidden_size]))
    hidden = tf.nn.sigmoid(tf.matmul(initial_state, H) + tf.matmul(inputs[0], I) + b1)
    rnn_outputs = [hidden]
    for input in inputs[1:]:
        scope.reuse_variables()
        hidden = tf.nn.sigmoid(tf.matmul(hidden, H) + tf.matmul(input, I) + b1)
        rnn_outputs.append(hidden)

In [53]:
hidden

<tf.Tensor 'RNNffebraze/Sigmoid_9:0' shape=(64, 100) dtype=float32>

In [54]:
rnn_outputs

[<tf.Tensor 'RNNffebraze/Sigmoid:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_1:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_2:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_3:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_4:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_5:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_6:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_7:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_8:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'RNNffebraze/Sigmoid_9:0' shape=(64, 100) dtype=float32>]

In [ ]:
tf.global_variables_initializer